In [1]:
import os
import numpy as np
import pylab
import imageio
from matplotlib import pyplot as plt
import cv2
from os.path import isfile, join

In [2]:
print(os.listdir('.'))

['.ipynb_checkpoints', 'Bebop2_20180414154341-0700.mp4', 'Frames', 'Semantic_Segmentation.ipynb', 'Video Frame Extraction with Function Placeholders.ipynb', 'Video Frame Extraction.ipynb']


**Module to operate on each individual frame of the video**

In [3]:
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])

In [4]:
def ProcessChip (frame):
    
    result_feature_map = np.zeros((9,16,7))  #CNN feature map to be returned
    
    for i in range(0,9):
        for j in range(0,16):
            
            chip = frame[120*i:120*(i+1), 120*j:120*(j+1), :]
            
            #Send chip to CNN or any other operation by calling the function here
            #Eg: result = CNN(chip). 
            
            #Uncomment next line to build feature map
            #result_feature_map[i,j,:] = result 
    
    #Uncomment next line when you run CNN through all the chips.
    #return result_feature_map
    
    #For now I'm just checking if the function is iterating through the chips correctly.
    #Remove next line once CNN call is added. Returns the bottom-left chip of frame.
    return chip

In [5]:
def ProcessChip2 (frame):
    
    result_feature_map = np.zeros((9,16,7))  #CNN feature map to be returned
    chips = np.zeros((144,120,120,3))
    
    for i in range(0,9):
        for j in range(0,16):
            
            chips[16*i+j,:,:,:] = frame[120*i:120*(i+1), 120*j:120*(j+1), :]
            #chip = frame[120*i:120*(i+1), 120*j:120*(j+1), :]
    
    #return chips[143,:,:,:]
    return rgb2gray(frame)

**Module to iterate through each frame in video**

In [6]:
def VideoToFrames (vid):
    
    count = 0 # Can be removed. Just to verify number of frames
    
    for image in vid.iter_data(): #Iterate through every frame in Video
        #image: numpy array containing image information

        #Insert Function here to operate on frame. For now I've put my ProcessChip() placeholder.
        #ProcessChip is currently just returning a chip. Modify its return statement once CNN function call is added.
        frame_feature_map = ProcessChip(image)
        frame_feature_map_2 = ProcessChip2(image)
        #plt.imshow(frame_feature_map_2, interpolation='nearest', cmap = plt.get_cmap('gray'))
        #plt.show()
        #cv2.imwrite('./Frames/frame%d.jpg'%count, frame_feature_map_2)
        #Call the Drone Decision Module here. Will take the 9x16x7 CNN feature map stored in frame_feature_map as input

        #The following is to just check if frames are being read correctly. Can be removed.
        count += 1
        if count > 100:
            return
        if 1== 0: #Printing every 100th frame to verify
            #print (frame_feature_map_2.shape)
            print ('The %dth frame: '%count,'\n')
            plt.imshow(image, interpolation='nearest')
            plt.show()
            print ('Bottom Left Chip from the %dth frame: '%count,'\n') #Printing bottom left chip of every 100th frame
            plt.imshow(frame_feature_map, interpolation='nearest')
            plt.show()
            print ('Bottom Left Chip from the %dth frame using chips: '%count,'\n') #Printing bottom left chip of every 100th frame
            plt.imshow(frame_feature_map_2, interpolation='nearest')
            plt.show()
            
    return

**Module to convert Image Frames to Video**

In [7]:
def convert_frames_to_video(pathIn,pathOut,fps):
    frame_array = []
    files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
 
    #for sorting the file names properly
    files.sort(key = lambda x: int(x[5:-4]))
 
    for i in range(len(files)):
        filename=pathIn + files[i]
        #reading each file
        img = cv2.imread(filename)
        height, width, layers = img.shape
        size = (width,height)
        print(filename)
        #inserting the frames into an image array
        frame_array.append(img)
 
    out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
 
    for i in range(len(frame_array)):
        # writing to a image array
        out.write(frame_array[i])
    out.release()

In [8]:
filename = './Bebop2_20180414154341-0700.mp4' #Add path to video file

In [9]:
vid = imageio.get_reader(filename, 'ffmpeg') #You can use any reader of your choice
#print (vid.iter_data())

In [10]:
VideoToFrames(vid) #Passing the video to be analyzed frame by frame

**Frames to Video**

**Frames Verification** 

You can also put some sort of check in this in case you don't want the drone to execute an action. For now I'm just checking if the frames are read in correctly.

In [ ]:
count = 0
try:
    for _ in vid:
        count += 1
except RuntimeError:
    print('Something went wront in iterating, maybe wrong fps number?')
finally:
    print('Number of frames counted {} \nNumber of frames in metada {}'.format(count, vid.get_meta_data()['nframes']))